## EMPAQUETAR TODO EL PROCESO EN FUNCIONES

Vamos a refactorizar que consiste en coger todo lo que hemos hecho anteriormente y meterlo en funciones

### Calidad de datos

In [1]:
#Mirar excel para entender el proceso que se ha aplicado a cada vaiable, cambio de tipos,nulos etc 
def calidad_datos(x):
    
    #Modificar tipos
    temp = x.astype({'month': 'O', 'wday': 'O'})             
    
    #Imputar nulos
    temp.loc[x['event_name_1'].isna(),'event_name_1'] = 'Sin_evento'
    
    def imputar_moda(registros):
        #Calcula la moda del precio en ese producto
        moda = registros.sell_price.mode()[0]
        #Imputa los nulos
        registros.loc[registros.sell_price.isna(),'sell_price'] = moda
        #Devuelve todos los registros del producto
        return(registros)

    temp = temp.groupby('item_id').apply(imputar_moda)
      
    return(temp)

### Creación de variables

In [2]:
def crear_variables(x):
    
    #DEMANDA INTERMITENTE
    
    def rotura_stock(ventas, n = 5):
        cero_ventas = pd.Series(np.where(ventas == 0,1,0))
        num_ceros = cero_ventas.rolling(n).sum()
        rotura_stock = np.where(num_ceros == n,1,0)
        return(rotura_stock)
    
    x = x.sort_values(by = ['store_id','item_id','date'])
    x['rotura_stock_3'] = x.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x, 3)).values
    x['rotura_stock_7'] = x.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x,7)).values
    x['rotura_stock_15'] = x.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x,15)).values
    
    
    #LAGS
    
    def crear_lags(x, variable, num_lags = 7):
        lags = pd.DataFrame()
        for cada in range(1,num_lags+1):
            lags[variable + '_lag_'+ str(cada)] = x[variable].shift(cada)
        return(lags)
    
    lags_sell_price_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: crear_lags(x = x, variable = 'sell_price', num_lags= 7))
    
    lags_rotura_stock_3_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: crear_lags(x = x, variable = 'rotura_stock_3', num_lags= 1))
    
    lags_rotura_stock_7_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: crear_lags(x = x, variable = 'rotura_stock_7', num_lags= 1))
    
    lags_rotura_stock_15_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: crear_lags(x = x, variable = 'rotura_stock_15', num_lags= 1))
    
    lags_ventas_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: crear_lags(x = x, variable = 'ventas', num_lags= 15))
    
    
    #VENTANAS MÓVILES
    
    def min_movil(x, variable, num_periodos = 7):
        minm = pd.DataFrame()
        for cada in range(2,num_periodos+1):
            minm[variable + '_minm_' + str(cada)] = x[variable].shift(1).rolling(cada).min()
        return(minm)
    
    def media_movil(x, variable, num_periodos = 7):
        mm = pd.DataFrame()
        for cada in range(2,num_periodos+1):
            mm[variable + '_mm_' + str(cada)] = x[variable].shift(1).rolling(cada).mean()
        return(mm)
    
    def max_movil(x, variable, num_periodos = 7):
        maxm = pd.DataFrame()
        for cada in range(2,num_periodos+1):
            maxm[variable + '_maxm_' + str(cada)] = x[variable].shift(1).rolling(cada).max()
        return(maxm)
    
    min_movil_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: min_movil(x = x, variable = 'ventas', num_periodos= 15))
    
    media_movil_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: media_movil(x = x, variable = 'ventas', num_periodos= 15))
    
    max_movil_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: max_movil(x = x, variable = 'ventas', num_periodos= 15))
    
    
    #UNIR DATAFRAMES GENERADOS
    
    x_unido = pd.concat([x,
                      lags_sell_price_x,
                      lags_rotura_stock_3_x,
                      lags_rotura_stock_7_x,
                      lags_rotura_stock_15_x,
                      lags_ventas_x,
                      min_movil_x,
                      media_movil_x,
                      max_movil_x], axis = 1)

    x_unido.dropna(inplace=True)
    
    x_unido.drop(columns = ['sell_price','rotura_stock_3','rotura_stock_7','rotura_stock_15'],
                  inplace=True)
    
    #Crear una sola variable para el producto
    x_unido.insert(loc=0,column='producto',value=x_unido.store_id + '_'+ x_unido.item_id)
    x_unido = x_unido.drop(columns = ['store_id','item_id'])
    
    return(x_unido)

### Transformación de variables

In [3]:
def transformar_variables(x,y=None,modo = 'entrenamiento'):
    
    
    '''
    He modificado esta función para que sirva tanto para entrenamiento como para ejecución:

    * Incluyendo el parámetro modo, que por defecto es entrenamiento
    * Haciendo que el parámetro y sea opcional, ya que en ejecución no se usa

    Cuando se usa en modo entrenamiento aplica el método fit_transform y guarda los objetos.

    Mientras que cuando se usa en modo ejecución carga los objetos y aplica solo el método transform.
    '''    
    
    x.reset_index(inplace = True)

    #GESTION DE LOS ENCODERS
    nombre_ohe = 'ohe_retail.pickle'
    nombre_te = 'te_retail.pickle'
    ruta_ohe = ruta_proyecto + '/04_Modelos/' + nombre_ohe
    ruta_te = ruta_proyecto + '/04_Modelos/' + nombre_te
    
    #ONE HOT ENCODING
    var_ohe = ['event_name_1']
    if modo == 'entrenamiento':
        #Si está en entrenamiento aplica fit_transform y guarda el encoder
        ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')
        ohe_x = ohe.fit_transform(x[var_ohe])
        ohe_x = pd.DataFrame(ohe_x, columns = ohe.get_feature_names_out())
        with open(ruta_ohe, mode='wb') as file:
            pickle.dump(ohe, file)
    else:
        #Si está en ejecución recupera el guardado y solo aplica transform
        with open(ruta_ohe, mode='rb') as file:
            ohe = pickle.load(file)
        ohe_x = ohe.transform(x[var_ohe])
        ohe_x = pd.DataFrame(ohe_x, columns = ohe.get_feature_names_out())

    #TARGET ENCODING    
    var_te = ['month','wday','weekday']
    if modo == 'entrenamiento':
        #ASEGURAR QUE Y TIENE LOS MISMOS REGISTROS QUE X
        y.reset_index(inplace = True, drop = True)
        y = y.loc[y.index.isin(x.index)]
        #Si está en entrenamiento aplica fit_transform y guarda el encoder
        te = TargetEncoder(min_samples_leaf=100, return_df = False)
        te_x = te.fit_transform(x[var_te], y = y)
        nombres_te = [variable + '_te' for variable in var_te]
        te_x = pd.DataFrame(te_x, columns = nombres_te)
        with open(ruta_te, mode='wb') as file:
            pickle.dump(te, file)
    else:
        #Si está en ejecución recupera el guardado y solo aplica transform
        with open(ruta_te, mode='rb') as file:
            te = pickle.load(file)
        te_x = te.transform(x[var_te])
        nombres_te = [variable + '_te' for variable in var_te]
        te_x = pd.DataFrame(te_x, columns = nombres_te)
    
      
    #INTEGRAR, LIMPIAR Y DEVOLVER EL DATAFRAME
    #Eliminar las originales ya transformadas
    x = x.drop(columns=['event_name_1','month','wday','weekday'])
    #Incorporar los otros dataframes
    x = pd.concat([x,ohe_x,te_x], axis=1).set_index('date')

    #Salida
    return(x)

### Preselección de variables

In [4]:
def preseleccionar_variables(x,y):
    
    '''
    Sólo hay que usarla en entrenamiento.
    '''
    #ELIMINAR LA COLUMNA PRODUCTO Y EL INDEX
    x.reset_index(drop = True,inplace = True)
    x.drop(columns='producto',inplace = True)
    
    #ASEGURAR QUE Y TIENE LOS MISMOS REGISTROS QUE X
    y = y.loc[y.index.isin(x.index)]
    

    mutual_selector = mutual_info_regression(x,y)
    posicion_variable_limite = 70
    ranking_mi = pd.DataFrame(mutual_selector, index = x.columns).reset_index()
    ranking_mi.columns = ['variable','importancia_mi']
    ranking_mi = ranking_mi.sort_values(by = 'importancia_mi', ascending = False)
    ranking_mi['ranking_mi'] = np.arange(0,ranking_mi.shape[0])
    entran_mi = ranking_mi.iloc[0:posicion_variable_limite].variable
    x_mi = x[entran_mi].copy()

    return(x_mi)

### Modelización

Esta funcion recibe tanto los datos de las variables predictoras de un producto determinado como la target y de un producto determinado, lo que va a hacer es encontrar los parametros optimos para ese producto, y deveolver el mejor modelo.

Parte operativa 

1. sacamos variables que no sean de modelizacion var_modelizar = x_producto.columns.to_list()[2:]

2. definimos la validacion cruzada, aqui ya hacemos el modelo definitivo por lo tanto vamos a validar mas veces 
time_cv = TimeSeriesSplit(5, test_size = 8)

3. definimos parrila de algorimos que en este caso solo es 1 XGBoost pipe = Pipeline([('algoritmo',HistGradientBoostingRegressor())])

4. Creamos el randomsearch, en este caso no nos va a hacer ningun tipo de optimizacion para buscar los  mejores parametros porque nos va hacer el algoritmo por defecto ya que es la mejor solucion a nuestro problema.

5. Aqui en esta parte modelo = random_search.fit(x_producto[var_modelizar],y) lo que va hacer es entrenar cual es ese modelo,sobre la configuracion de random_search que le hemos pasado.

6. Seleccion un mejor modelo modelo_final = modelo.best_estimator_.fit(x_producto[var_modelizar],y), cogera el modelo definitivo

7. El retorno de esta funcion es el modelo defintivo



#### Entrenamiento

In [5]:

def modelizar(x_producto, y):
    
    '''
    Esta función es la que hace la modelización individual.

    Recibe los datos de las x y la y de un producto.

    Encuentra los parámetros óptimos para ese producto.

    Devuelve el mejor modelo.
    '''
      
    #Excluye el producto como variable de modelización
    var_modelizar = x_producto.columns.to_list()[2:]
    
    #Define la validación cruzada
    time_cv = TimeSeriesSplit(5, test_size = 8)
    
    #Define la parrilla de algoritmos
    pipe = Pipeline([('algoritmo',HistGradientBoostingRegressor())])
    
    grid = [ 
         {'algoritmo': [HistGradientBoostingRegressor()]
#          'algoritmo__learning_rate': [0.01,0.025,0.05,0.1],
#          'algoritmo__max_iter': [50,100,200],
#          'algoritmo__max_depth': [5,10,20,50],
#          'algoritmo__scoring': ['neg_mean_absolute_error'],
#          'algoritmo__l2_regularization': [0,0.25,0.5,0.75,1]
         }
                       
    ]
           
    #Crea los modelos
    random_search = RandomizedSearchCV(estimator = pipe,
                                   param_distributions = grid, 
                                   n_iter = 1, 
                                   cv = time_cv, 
                                   scoring = 'neg_mean_absolute_error', 
                                   verbose = 0,
                                   n_jobs = -1)
    
    modelo = random_search.fit(x_producto[var_modelizar],y)
    
    #Reentrena el mejor sobre todos los datos
    modelo_final = modelo.best_estimator_.fit(x_producto[var_modelizar],y)
    
    #Devuelve como salida el modelo final
    return(modelo_final)

Funcion lanzar_entrenamiento(df):

Esta función va recorriendo todos los productos y llamando a modelizar() para crear una lista total con todos los modelos de todos los productos.

1. Recibe el df ya limpio y segmentado por producto y tambien la target
2. Creacion de la lista de los productos para despues recorrerla
3. Lista vacia para guardar los que vayamos entrenando.
4. separamos target y variables predictoras
5. aplicar la funcion de transformacion_variables tanto a columnas como y
6. seguido de preseleccionar variables tanto de target como de y
7. llamamos a la funcion modelizar y le pasamos predictoras y target
8. añadimos ese modelo final por producto en nuestra lista de modelos.
9. Despues de recorrer el bucle creamos un nombre para el archivo pickle donde vamos a guardar la lista de nuestros modelos
entrenados


In [6]:
def lanzar_entrenamiento(df):
    
    '''
    Esta función va recorriendo todos los productos y llamando a modelizar() para crear una lista total con todos los modelos de todos los productos.

    Recibe el dataframe de las x ya limpio y segmentado por producto, y también la target.

    No devuelve nada, si no que guarda en disco el objeto (lista donde estan todos los modelos) ya entrenado con todos los modelos.
    '''
    
    lista_productos = list(df.producto.unique())
    
    lista_modelos =[] 
    
    for cada in lista_productos:
        
        #Renombra por claridad
        producto = cada
        target = 'ventas'

        x = df.loc[df.producto == producto].copy().drop(columns=target).copy()
        y = df.loc[df.producto == producto,'ventas'].copy()

        x = transformar_variables(x,y)
        x = preseleccionar_variables(x,y)
        
        #Llama a la funcion de modelizar
        modelo = modelizar(x,y)
        
        #Añade el modelo final a la lista
        lista_modelos.append((producto,modelo))
        
    #Guarda la lista de modelos entrenados
    nombre_modelos = 'lista_modelos_retail.pickle'
    ruta_modelos = ruta_proyecto + '/04_Modelos/' + nombre_modelos
    with open(ruta_modelos, mode='wb') as file:
        pickle.dump(lista_modelos, file)

#### Ejecución

En esta fase cambia la mentalidad ya no estamos en entrenamiento sino que estamos en ejecucion, por lo tanto aqui tendremos que pasarle los datos no a 3 años que teniamos informacion sino a los dias que querramos predecir 7,8,9,10 dias lo que sea.

Esta funcion recibe un dataframe y lo que va hacer es el forecast pero de un dia.

Tiene que tener la estructura del fichero DatosParaProduccion.csv de la carpeta Validación, es decir,tenemos que tener datos diferentes a los que teniamos enentrenamiento puesto que ahora estamos en producccion y va a recibir unos datos nuevos.

Este sistema no va recibir informacion a 3 años, sino a menos tiempo, hacia atras.Por lo tanto nuestro fichero tiene que tener la estrucutra de los datos del tiempo que necesitamos pasar.


Esta funcion devuele la produccion para todos los productos para el dia que toca.

La operativa de la funcion es parecida a la parte del notebook 06_Modelizacion

In [17]:
#Lanzar la predicción
def lanzar_ejecucion(df):
    
    '''
    Esta función hace el forecast para cada producto, pero solo de un día.

    Recibe el nuevo dataset a predecir.
    
    Que tiene que tener la estructura del fichero DatosParaProduccion.csv de la carpeta Validación.

    Va recorriendo cada producto, cargando su modelo correspondiente, seleccionando sus datos, y haciendo las predicciones.

    Devuelve la predicción para todos los productos pero SOLO PARA EL DÍA QUE TOCA.
    
    
    '''
    
    #CARGA LOS MODELOS
    nombre_modelos = 'lista_modelos_retail.pickle'
    ruta_modelos = ruta_proyecto + '/04_Modelos/' + nombre_modelos
    with open(ruta_modelos, mode='rb') as file:
        lista_modelos = pickle.load(file)
    
    predicciones_df = pd.DataFrame(columns=['date','producto','ventas','prediccion'])
    
    for cada in range(0,len(lista_modelos)):

        producto = lista_modelos[cada][0]
        modelo = lista_modelos[cada][1]
        
#ojo, como estamos en ejecucion no vamos a llamar a la varaible de preseleccion de variables sino que la variable
#que ha entrado en cada modelo y que necesitamos recuperar estan guardadas en el atributo feature_names_in_ del propio modelo
        variables = modelo[0].feature_names_in_
        target = 'ventas'
        
        #separacion predictoras y target
        x = df.loc[df.producto == producto].copy().drop(columns=target).copy()
        y = df.loc[df.producto == producto,'ventas'].copy()
        
        #preparacion de la informacion
        date = df.reset_index().copy()
        date = date.loc[date.producto == producto,'date'].values

        #Transformacion de variables
        #Porque necesitamos una transformacion de variables?
        #Porque esta funcion va recibir el df despues de haber pasado por la fase de creacion de variables
        #pero todavia no ha pasado por la fase de transformacion de variable.
        #Lo que pasa es que el modelo va a estar esperando la variable en su formato de OHE o TE pero todavia no esta
        #en ese formato, esa variable esta en formato original, por eso llamamos a la funcion
        #lo ponemos en modo ejecucion, por tanto, ya no va identificar que valores tiene la variable original
        #no va calcular las medias para cada uno de los valores en el target encoding, todo eso ya esta hecho
        #ahora va recuperar esa informacion y aplicarla para devolverme las variables en el mismo formato en el que fueron entrenadas
        #y por lo tanto que el modelo funcione
        x = transformar_variables(x, modo = 'ejecucion')
        
        #Seleccion de variables
        #OJO, se va quedar solo con las variables que espera para ese producto en concrecto, porque si el modelo que estamos utilizando
        #en un producto determinado,que es este modelo = lista_modelos[cada][1], espera las variables 3 4 y 5 no va funcionar va dar error
        #por eso necesitamos seleccionar para cada uno de los productos las variables y unicamente las variables, con las que ese
        #modelo fue entrenado en su version final, esto es clave en modeliacion masiva
        # Conclusion: al haber entrenado los 20 modelos, cada modelo con sus variables, es neecesario recoger las variables 
        #de cada uno de los modelos,ya que por cada producto hay variables diferentes, eso es lo que recogemos aqui
        x = x[variables]
        
        #Cálculo de predicciones
        #rellenamos el dataframe vacio, que hemos construido antes, con la fecha, producto,dato real ventas y prediccion
        predicciones = pd.DataFrame(data={'date': date,
                                          'producto': producto,
                                          'ventas': y,
                                          'prediccion': modelo.predict(x)})

        #esta parte da formato a este dataframe
        predicciones['prediccion'] = predicciones.prediccion.astype('int')

        predicciones_df = pd.concat([predicciones_df,predicciones])
        
        #en este punto vuelve a ejecutar este bucle tantas veces como productos tengamos,en este caso 20 veces
        #y una vez hayamos tenido esee dataframe relleno con todos los datos de todas las predicciones, de todos los productos
        #para todas las fechas, me va devolver lo relativo a una de esas fechas que es la linea de codigo de abajo
    
    #esta prediccion devuelve un dato el de un dia
    predicciones_df = predicciones_df.loc[predicciones_df.index == predicciones_df.index.min()]
    return(predicciones_df)

Se procede a explicar las dos maneras que tenemos para poder solucionar este problema de que los modelos de ML unicamente predicen a 1 dia vista,mientras que en las situaciones de negocio muchas veces, necsitamos predecir a varios dias vista.
Existen 2 aproximaciones, nosotros vamos a aplicar 1 aproximacion que es la recursiva.


In [20]:
def forecast_recursivo(x):
    
    '''
    Esta función es la que aplica el forecast recursivo para predecir 8 días.

    Recibe el nuevo dataset a predecir.(no los datos historicos, sino que los datos que estan en ejecucion con el formato
    que esperamos para ejecucion) 

    Que tiene que tener la estructura del fichero DatosParaProduccion.csv de la carpeta Validación. 
    
    Ya que para aplicar la recursividad (mirar leccion 37 para entender esta aproximacion):

    * Va a predecir el primer día para el cual tenga toda la información(concepto clave) (es decir 15 días desde el día más antiguo)
    * Al finalizar graba la predicción de ventas en el registro a predecir y elimina los registros del día más antiguo del dataframe
    * Por tanto en la siguiente iteración va a predecir el siguiente día.

    Por ejemplo:

    Si el día más antiguo del dataset es el 09/12/2015 entonces el primer día que puede predecir
    
    (y del cual ya no tenemos dato) es el 24/12/2015.

    Cuando predice el dato del 24 para cada producto lo sobrescribe como sus ventas
    
    y elimina todos los registros del día 09.

    Entonces el día más antiguo pasa a ser el día 10 y por tanto el día a predecir es el 25.

    Y así hasta que finaliza 8 ciclos para predecir la semana que queremos.
    '''
    
    for cada in range(0,8): #predecimos a 8 dias vista
        paso1_df = calidad_datos(x)
        paso2_df = crear_variables(paso1_df)
        
        #Calcula la predicción
        f = lanzar_ejecucion(paso2_df)
        f['store_id'] = f.producto.str[:4]
        f['item_id'] = f.producto.str[5:]

        #Actualiza el dato de ventas con la predicción
        x.loc[(x.index.isin(f.date)) & (x.store_id.isin(f.store_id)) & (x.item_id.isin(f.item_id)),'ventas'] = f.prediccion
                                                              
        #Elimina el día más antiguo de x
        x = x.loc[x.index != x.index.min()]
        
    return(x)

## PROCESO

### REENTRENAMIENTO

In [23]:
import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

from sklearn.feature_selection import mutual_info_regression

from sklearn.model_selection import TimeSeriesSplit

from sklearn.pipeline import Pipeline

from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

#Cargar los datos
ruta_proyecto = 'C:/Users/Marius/EstructuraDirectorio/03_MACHINE_LEARNING/06_CASOS/02_RETAIL'
nombre_fichero_datos = 'trabajo.csv'
ruta_completa = ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_fichero_datos
df = pd.read_csv(ruta_completa,sep=',',parse_dates=['date'],index_col='date')

#Seleccionar solo las que se han usado
variables_finales = ['store_id',
                     'item_id',
                     'event_name_1',                     
                     'month',
                     'sell_price',                      
                     'wday',
                     'weekday',
                     'ventas']

df = df[variables_finales]

paso1_df = calidad_datos(df)
paso2_df = crear_variables(paso1_df)

lanzar_entrenamiento(paso2_df)

### EVALUACIÓN

In [26]:
import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

from sklearn.feature_selection import mutual_info_regression

from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

#Cargar los datos
ruta_proyecto = 'C:/Users/Marius/EstructuraDirectorio/03_MACHINE_LEARNING/06_CASOS/02_RETAIL'
nombre_fichero_datos = 'validacion.csv'
ruta_completa = ruta_proyecto + '/02_Datos/02_Validacion/' + nombre_fichero_datos
df = pd.read_csv(ruta_completa,sep=',',parse_dates=['date'],index_col='date')

#Seleccionar solo las que se han usado
variables_finales = ['store_id',
                     'item_id',
                     'event_name_1',                     
                     'month',
                     'sell_price',                      
                     'wday',
                     'weekday',
                     'ventas']


df = df[variables_finales]

paso1_df = calidad_datos(df)
paso2_df = crear_variables(paso1_df)

forecast_1dia = lanzar_ejecucion(paso2_df) #utilizamos esta funcion y no la de forecast_recursivo,
#porque esta funcion devuelve a 1 solo dia

print('MAE = ', mean_absolute_error(forecast_1dia.ventas,forecast_1dia.prediccion))

forecast_1dia

#para arreglar el tema de la prediccion negativa, lo suyo seria pasarlo por calidad de datos, 
#la salida de la produccion decirle que si el registro es -X , poner esa prediccion a 0

MAE =  5.65


,date,producto,ventas,prediccion
2015-12-16,2015-12-16 00:00:00,CA_3_FOODS_3_090,0,-10
2015-12-16,2015-12-16 00:00:00,CA_3_FOODS_3_120,52,55
2015-12-16,2015-12-16 00:00:00,CA_3_FOODS_3_202,20,12
2015-12-16,2015-12-16 00:00:00,CA_3_FOODS_3_252,36,35
2015-12-16,2015-12-16 00:00:00,CA_3_FOODS_3_288,35,22
2015-12-16,2015-12-16 00:00:00,CA_3_FOODS_3_329,64,40
2015-12-16,2015-12-16 00:00:00,CA_3_FOODS_3_555,30,28
2015-12-16,2015-12-16 00:00:00,CA_3_FOODS_3_586,76,63
2015-12-16,2015-12-16 00:00:00,CA_3_FOODS_3_587,29,33
2015-12-16,2015-12-16 00:00:00,CA_3_FOODS_3_714,19,16


### EJECUCIÓN

Lección 41 -..>> como crear el fichero de DatosParaProducción.csv

In [25]:
import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

from sklearn.feature_selection import mutual_info_regression

from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

#Cargar los datos
ruta_proyecto = 'C:/Users/Marius/EstructuraDirectorio/03_MACHINE_LEARNING/06_CASOS/02_RETAIL'
nombre_fichero_datos = 'DatosParaProduccion.csv'
ruta_completa = ruta_proyecto + '/02_Datos/02_Validacion/' + nombre_fichero_datos
df = pd.read_csv(ruta_completa,sep=';',parse_dates=['date'],index_col='date')

#Seleccionar solo las que se han usado
variables_finales = ['store_id',
                     'item_id',
                     'event_name_1',                     
                     'month',
                     'sell_price',                      
                     'wday',
                     'weekday',
                     'ventas']

df = df[variables_finales]

#Lanzar la predicción
forecast = forecast_recursivo(df)

forecast.sort_values(by = ['store_id','item_id'])

,store_id,item_id,event_name_1,month,sell_price,wday,weekday,ventas
date,,,,,,,,
2015-12-17,CA_3,FOODS_3_090,NaN,12,1.00,6,Thursday,0
2015-12-18,CA_3,FOODS_3_090,NaN,12,1.00,7,Friday,1
2015-12-19,CA_3,FOODS_3_090,NaN,12,1.00,1,Saturday,0
2015-12-20,CA_3,FOODS_3_090,NaN,12,1.00,2,Sunday,6
2015-12-21,CA_3,FOODS_3_090,NaN,12,1.00,3,Monday,4
...,...,...,...,...,...,...,...,...
2015-12-27,CA_4,FOODS_3_714,NaN,12,1.58,2,Sunday,9
2015-12-28,CA_4,FOODS_3_714,NaN,12,1.58,3,Monday,6
2015-12-29,CA_4,FOODS_3_714,NaN,12,1.58,4,Tuesday,7
